Importy bibliotek

In [46]:
from dimacs import *
import networkx as nx 
from networkx.algorithms.planarity import check_planarity
from networkx.algorithms.flow import maximum_flow
from networkx.algorithms.components import strongly_connected_components
from networkx.algorithms.dag import topological_sort

Budowa grafu

In [47]:
def buildGraph(G):
    (V, L) = loadWeightedGraph(G)
    G = nx.Graph()
    for u, v, _ in L:
        G.add_edge(u, v)

    return G

Zadanie 1. Sprawdzanie planarności

In [48]:
for graph in os.listdir("graphs-lab7/planitarity"):
    G = buildGraph(f"graphs-lab7/planitarity/{graph}")
    print(f'Planarity of {graph}: {check_planarity(G)[0]}')

Planarity of 16-cell.txt: False


Zadanie 2. Maksymalny przepływ

In [49]:
def buildGraph(G):
    (V, L) = loadWeightedGraph(G)
    G = nx.Graph()
    for u, v, w in L:
        G.add_edge(u, v)
        G[u][v]['capacity'] = w
        
    return G

In [50]:
for graph in os.listdir("graphs-lab7/flow"):
    G = buildGraph(f"graphs-lab7/flow/{graph}")
    print(f'Maximum flow for {graph}: {maximum_flow(G, 1, len(G))[0]}')

Maximum flow for clique100: 5185
Maximum flow for clique20: 824
Maximum flow for clique5: 138
Maximum flow for grid100x100: 5080
Maximum flow for grid5x5: 22
Maximum flow for pp100: 136
Maximum flow for rand100_500: 2430
Maximum flow for rand20_100: 132
Maximum flow for simple: 24
Maximum flow for simple2: 6
Maximum flow for trivial: 3
Maximum flow for trivial2: 7
Maximum flow for worstcase: 20


Zadanie 3. SAT-2CNF

In [51]:
def buildImplicationGraph(F):
    (V, L) = loadCNFFormula(F)
    graph = nx.DiGraph()
    for clause in L:
        x, y = clause
        # Rownowazne alternatywie
        graph.add_edge(-x, y)
        graph.add_edge(-y, x)
        
    return graph


In [52]:
def isSatisfiable(G):
    scc = list(strongly_connected_components(G))
    for component in scc:
        for node in component:
            if -node in component:
                return False, _

    return True, scc

In [53]:
def assignValues(F):

    i_G = buildImplicationGraph(F)
    satisfiable, scc = isSatisfiable(i_G)

    if not satisfiable:
        return

    # Tworzenie grafu H silnie spójnych składowych
    H = nx.DiGraph()
    component_map = {node: comp for comp, nodes in enumerate(scc) for node in nodes}

    for u, v in i_G.edges():
        if component_map[u] != component_map[v]:
            H.add_edge(component_map[u], component_map[v])

    order = topological_sort(H)


    # Przydzielenie wartości zmiennym
    value_assignment = {}
    for component in order:
        for node in scc[component]:
            if node not in value_assignment and -node not in value_assignment:
                value_assignment[node] = False
                value_assignment[-node] = True

    return value_assignment

In [54]:
def checkAssingment(F, assingment):
    if not assingment:
        return
        
    for clause in F:
        x, y = clause
        logicalValue = assingment[x] or assingment[y]
        if not logicalValue:
            return False

    return True

In [55]:
for formula in os.listdir("graphs-lab7/sat"):
    F = loadCNFFormula(f"graphs-lab7/sat/{formula}")[1]
    assignment = assignValues(f"graphs-lab7/sat/{formula}")
    assignment_check = checkAssingment(F, assignment)
    if assignment_check is None:
        assignment_check = False
    print(f'Formula {formula} is satisfiable: {assignment_check}')

['c', 'solution=1']
['c', 'solution=1']
Formula sat100_100 is satisfiable: True
['c', 'solution=1']
['c', 'solution=1']
Formula sat100_150 is satisfiable: True
['c', 'solution=0']
['c', 'solution=0']
Formula sat100_200 is satisfiable: False
['c', 'solution=0']
['c', 'solution=0']
Formula sat100_250 is satisfiable: False
['c', 'solution=0']
['c', 'solution=0']
Formula sat100_400 is satisfiable: False
['c', 'solution=1']
['c', 'solution=1']
Formula sat15_30 is satisfiable: True
['c', 'solution=0']
['c', 'solution=0']
Formula sat30_100 is satisfiable: False
['c', 'solution=1']
['c', 'solution=1']
Formula sat30_50 is satisfiable: True
['c', 'solution=1']
['c', 'solution=1']
Formula sat5_10 is satisfiable: True
['c', 'solution=0']
['c', 'solution=0']
Formula sat5_20 is satisfiable: False
['c', 'solution=1']
['c', 'solution=1']
Formula simple_sat is satisfiable: True
['c', 'solution', '=', '0']
['c', 'solution', '=', '0']
Formula simple_unsat is satisfiable: False
